In [11]:
import numpy as np
import pandas as pd
from os import path
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.callbacks import EarlyStopping
from collections import Counter
import sklearn
from tensorflow.keras.callbacks import LearningRateScheduler

from sklearn import preprocessing, metrics
from sklearn.metrics import (roc_curve, auc, accuracy_score, precision_score, 
                             recall_score, f1_score, balanced_accuracy_score, 
                             matthews_corrcoef)
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, LabelBinarizer
import shap
from imblearn.over_sampling import RandomOverSampler
import innvestigate

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
np.random.seed(0)

tf.compat.v1.disable_eager_execution()

In [12]:
def confusion_metrics (name_model,predictions,true_labels):

    name = name_model
    pred_label = predictions
    y_test_01 = true_labels 

    with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print(name, file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    # pred_label = label[ypred]

    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)



    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)


    # voting_acc_01 = Acc
    # voting_pre_01 = Precision
    # weighed_avg_rec_01 = Recall
    # weighed_avg_f1_01 = F1
    # weighed_avg_bacc_01 = BACC
    # weighed_avg_mcc_01 = MCC
    # with open(output_file_name, "a") as f:print('Accuracy total: ', Acc,file=f)
    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)

    return Acc, Precision, Recall, F1, BACC, MCC

In [13]:
output_file_name = "CIC_Base.txt"


In [14]:

req_cols = [
    
    ' Packet Length Std', ' Total Length of Bwd Packets', ' Subflow Bwd Bytes',
    ' Destination Port', ' Packet Length Variance', ' Bwd Packet Length Mean',' Avg Bwd Segment Size',
    'Bwd Packet Length Max', ' Init_Win_bytes_backward','Total Length of Fwd Packets',
    ' Subflow Fwd Bytes', 'Init_Win_bytes_forward', ' Average Packet Size', ' Packet Length Mean',
    ' Max Packet Length',
    ' Down/Up Ratio', ' Fwd URG Flags', ' Flow IAT Std', 'Subflow Fwd Packets', ' Flow Packets/s', ' URG Flag Count', 'FIN Flag Count', ' Bwd Packets/s', 'Bwd Avg Bulk Rate'
    , ' act_data_pkt_fwd', ' Fwd Packet Length Std', ' Bwd Avg Bytes/Bulk', ' Active Max', ' Flow IAT Max', ' min_seg_size_forward', ' Bwd Packet Length Std', ' Fwd IAT Std', ' Fwd Avg Bulk Rate', ' Fwd Packet Length Mean', ' Fwd Packet Length Max', ' Idle Std', ' CWE Flag Count', 'Fwd IAT Total'
    
    , ' ACK Flag Count', ' Bwd URG Flags', ' Flow IAT Min', ' Flow IAT Mean', ' Total Backward Packets', ' Fwd Avg Packets/Bulk', 'Fwd Avg Bytes/Bulk', ' SYN Flag Count', ' Min Packet Length', ' Fwd Packet Length Min', 'Idle Mean', 'Fwd PSH Flags', ' Fwd IAT Min'
     
    ,  ' Fwd Header Length', ' RST Flag Count', ' Idle Max', ' PSH Flag Count', ' Bwd Header Length', ' ECE Flag Count', ' Subflow Bwd Packets', 'Active Mean', 'Flow Bytes/s', ' Bwd IAT Mean', ' Avg Fwd Segment Size', ' Bwd Packet Length Min', ' Active Std', ' Bwd IAT Min', ' Flow Duration', 'Fwd Packets/s', ' Fwd IAT Max', 'Bwd IAT Total', ' Idle Min', ' Bwd PSH Flags', ' Bwd Avg Packets/Bulk', ' Total Fwd Packets', ' Active Min', ' Bwd IAT Std', ' Fwd IAT Mean', ' Bwd IAT Max'
            
            , ' Label']

#-------------------------------------
#Load Databases from csv file

path_str = '/home/oarreche@ads.iu.edu/HITL/cicids/cicids_db/'
fraction = 1
#---------------------------------------------------------------------
#Load Databases from csv file
print('---------------------------------------------------------------------------------')
print('Loading Databases')
print('---------------------------------------------------------------------------------')
print('')


df0 = pd.read_csv (path_str + 'Wednesday-workingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)

df1 = pd.read_csv (path_str + 'Tuesday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df2 = pd.read_csv (path_str +'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df3 = pd.read_csv (path_str +'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df4 = pd.read_csv (path_str +'Monday-WorkingHours.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df5 = pd.read_csv (path_str +'Friday-WorkingHours-Morning.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df6 = pd.read_csv (path_str +'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)


df7 = pd.read_csv (path_str +'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', usecols=req_cols).sample(frac = fraction)

frames = [df0, df1, df2, df3, df4, df5, df6, df7]

df = pd.concat(frames,ignore_index=True)

df = df.sample(frac = 0.333)

# df = df.sample(frac = 0.001)


#---------------------------------------------------------------------
# Normalize database
print('---------------------------------------------------------------------------------')
print('Normalizing database')
print('---------------------------------------------------------------------------------')
print('')

df_max_scaled = df.copy()

y = df_max_scaled[' Label'].replace({'DDoS' :'Dos/Ddos' ,'DoS GoldenEye': 'Dos/Ddos', 'DoS Hulk': 'Dos/Ddos', 'DoS Slowhttptest': 'Dos/Ddos', 'DoS slowloris': 'Dos/Ddos', 'Heartbleed': 'Dos/Ddos','FTP-Patator': 'Brute Force', 'SSH-Patator': 'Brute Force','Web Attack - Brute Force': 'Web Attack', 'Web Attack - Sql Injection': 'Web Attack', 'Web Attack - XSS': 'Web Attack'})

df_max_scaled.pop(' Label')


df_max_scaled
for col in df_max_scaled.columns:
    t = abs(df_max_scaled[col].max())
    df_max_scaled[col] = df_max_scaled[col]/t
df_max_scaled
df = df_max_scaled.assign( Label = y)
#df
df = df.fillna(0)

#---------------------------------------------------------------------





---------------------------------------------------------------------------------
Loading Databases
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------
Normalizing database
---------------------------------------------------------------------------------



In [15]:
# Separate features and labels 
print('---------------------------------------------------------------------------------')
print('Separating features and labels')
print('---------------------------------------------------------------------------------')
print('')

y = df.pop('Label')
X = df
# summarize class distribution
counter = Counter(y)
print(counter)

df = X.assign( Label = y)

y, label = pd.factorize(y)

# Separate Training and Testing db

---------------------------------------------------------------------------------
Separating features and labels
---------------------------------------------------------------------------------

Counter({'BENIGN': 756652, 'Dos/Ddos': 127047, 'PortScan': 52987, 'Brute Force': 4594, 'Web Attack': 716, 'Bot': 628, 'Infiltration': 13})


In [16]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
import pandas as pd

# Assume X is your feature matrix and y is the target variable
k = 20  # Number of top features to select

# Use SelectKBest with mutual_info_classif as the scoring function
selector = SelectKBest(score_func=mutual_info_classif, k=k)
X_new = selector.fit_transform(X, y)

# Get the scores and the selected feature names
scores = selector.scores_
selected_features = selector.get_support(indices=True)
selected_feature_names = X.columns[selected_features]

# Convert to a Pandas DataFrame for better visualization
k_best_df = pd.DataFrame({'Feature': selected_feature_names, 'Score': scores[selected_features]})
k_best_df = k_best_df.sort_values(by='Score', ascending=False)

# Display top features based on SelectKBest
print(k_best_df.to_string())  # Ensures no truncation

                         Feature     Score
14           Average Packet Size  0.478872
13        Packet Length Variance  0.476843
12             Packet Length Std  0.476447
11            Packet Length Mean  0.450019
0               Destination Port  0.431400
3    Total Length of Bwd Packets  0.423641
17             Subflow Bwd Bytes  0.423626
15          Avg Bwd Segment Size  0.413709
6         Bwd Packet Length Mean  0.413208
5          Bwd Packet Length Max  0.387337
16             Subflow Fwd Bytes  0.384109
2    Total Length of Fwd Packets  0.383956
19       Init_Win_bytes_backward  0.383346
18        Init_Win_bytes_forward  0.381655
10             Max Packet Length  0.355453
4          Fwd Packet Length Max  0.343933
7                   Flow IAT Max  0.332244
9                  Bwd Packets/s  0.323523
1                  Flow Duration  0.317895
8                    Fwd IAT Max  0.311460


In [ ]:
#                          Feature     Score
# 14           Average Packet Size  0.478872
# 13        Packet Length Variance  0.476843
# 12             Packet Length Std  0.476447
# 11            Packet Length Mean  0.450019
# 0               Destination Port  0.431400
# 3    Total Length of Bwd Packets  0.423641
# 17             Subflow Bwd Bytes  0.423626
# 15          Avg Bwd Segment Size  0.413709
# 6         Bwd Packet Length Mean  0.413208
# 5          Bwd Packet Length Max  0.387337
# 16             Subflow Fwd Bytes  0.384109
# 2    Total Length of Fwd Packets  0.383956
# 19       Init_Win_bytes_backward  0.383346
# 18        Init_Win_bytes_forward  0.381655
# 10             Max Packet Length  0.355453
# 4          Fwd Packet Length Max  0.343933
# 7                   Flow IAT Max  0.332244
# 9                  Bwd Packets/s  0.323523
# 1                  Flow Duration  0.317895
# 8                    Fwd IAT Max  0.311460